In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb

import random
from collections import Counter, defaultdict

In [2]:
def stratified_group_k_fold(X, y, groups, k, seed=None):
    np.random.seed(seed)
    # ラベルの数をカウント
    labels_num = np.max(y) + 1
    # 各グループのラベルの数をカウントする
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1
    # 各フォールドのラベルの数をカウント
    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)

    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)

    for i in range(k):
        test_k = i
        # val_k = i+1 if i+1 != k else 0
        # print(val_k)
        train_groups = all_groups - groups_per_fold[test_k]  #  - groups_per_fold[val_k]
        # val_groups = groups_per_fold[val_k]
        test_groups = groups_per_fold[test_k]
        # print(test_groups)
        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        # val_indices = [i for i, g in enumerate(groups) if g in val_groups]
        # test_indices = {str(g): [i for i, g in enumerate(groups) if g in test_groups]}

        test_indices = []
        n_g = None
        test_list = []
        for i, g in enumerate(groups):
            if g in test_groups:
                if n_g is not None and n_g != g:
                    test_indices.append(test_list)
                    test_list = []
                test_list.append(i)
                n_g = g

        test_indices = [np.random.choice(i) for i in test_indices]
        yield train_indices, test_indices  # val_indices,

In [3]:
train_path = '/code/data/processed/proceeded_train_20200115_215607.csv'
test_path = '/code/data/processed/proceeded_test_20200115_215607.csv'

train = pd.read_csv(train_path, index_col=0)
test = pd.read_csv(test_path, index_col=0)

In [0]:
train.head()

In [0]:
train['target'] = 0
test['target'] = 1

In [0]:
test['target']

In [0]:
train_df = pd.concat([train, test])
train_df.columns = train_df.columns.str.replace(',', '')

In [0]:
train_df.shape

In [0]:
y = train_df['target']
x = train_df.drop('target', axis=1)
groups = np.array(x['installation_id'])
lgb_params = {
        'objective': 'binary',
        'metric': 'auc',
    }
x = x.drop(['accuracy_group', 'installation_id'], axis=1)

In [0]:
def lgb_regression(x, y, groups, lgb_params) -> pd.DataFrame:

    num_fold = 3
    pred = np.zeros(y.shape)
    all_importance = []

    for fold_ind, (train_ind, test_ind) in enumerate(
            stratified_group_k_fold(X=x, y=y, groups=groups, k=num_fold, seed=77)):
        x_train = x.iloc[train_ind]
        y_train = y.iloc[train_ind]
        x_test = x.iloc[test_ind]
        y_test = y.iloc[test_ind]
        lgb_train = lgb.Dataset(x_train, y_train)
        lgb_val = lgb.Dataset(x_test, y_test, reference=lgb_train)

        model = lgb.train(params=lgb_params,
                          train_set=lgb_train,
                          valid_sets=lgb_val)

        pred[test_ind] = model.predict(x_test, num_iteration=model.best_iteration)

        all_importance.append(pd.DataFrame(model.feature_importance('gain'), index=x_train.columns))

    all_importance = pd.concat(all_importance, axis=1)
    return pred, all_importance


In [0]:
pred, importance = lgb_regression(x, y, groups, lgb_params)

In [0]:
importance.mean(axis=1).sort_values(ascending=False).head(20)

In [0]:
"Air Show_3120","Flower Waterer (Activity)_4080","Bubble Bath_2035","Air Show_2075","Chow Time_4095","Crystals Rule_2010","Scrub-A-Dub_4090","Mushroom Sorter (Assessment)_3010","Crystals Rule_2030","Crystals Rule_3010","Chow Time_4090","Cauldron Filler (Assessment)_2010","Bubble Bath_4020","Chow Time_2020","Magma Peak - Level 1_2000","Bottle Filler (Activity)_4030","Lifting Heavy Things_2000","Bird Measurer (Assessment)_4020","Cauldron Filler (Assessment)_4100","Chicken Balancer (Activity)_4022"," Bird Measurer (Assessment)_3021"," Dino Drink_2060"," Mushroom Sorter (Assessment)_4100"" Cart Balancer (Assessment)_4035"," Dino Drink_3021"," Cart Balancer (Assessment)_4030"," Watering Hole (Activity)_3010","Pan Balance_4010"," Chicken Balancer (Activity)_4070"," Scrub-A-Dub_2081"," Scrub-A-Dub_4010"," Dino Drink_3010"," All StarSorting_4020"," Cauldron Filler (Assessment)_3010"," Leaf Leader_2020"," Scrub-A-Dub_4080"," Cart Balancer (Assessment_3120"," Bird Measurer (Assessment)_4040"," Flower Waterer (Activity)_4025"," Bird Measurer (Assessment)_3010"," DinoDive_4090"," Happy Camel_2081"," Scrub-A-Dub_3021"," Leaf Leader_4090"," Leaf Leader_3121"," Chest Sorter (Assessment)_3120"" Dino Dive_4080"," Bottle Filler (Activity)_2020"," Leaf Leader_4010"," Air Show_3020, Watering Hole (Activity)_3110, Cauldron Filler (Assessment)_3120, Pan Balance_4020, Bubble Bath_4230, Bubble Bath_3120, Bubble Bath_3010, Happy Camel_4095, Chest Sorter (Assessment)_4040, Pan Balance_4090, Air Show_3110, Pan Balance_3021, Mushroom Sorter (Assessment)_2035, Air Show_3010, Fireworks (Activity)_4030, Chest Sorter (Assessment)_2010, Bubble Bath_2030, All Star Sorting_2030, Happy Camel_4010, Mushroom Sorter (Assessment)_4090, Air Show_3121, All Star Sorting_3121, Bug Measurer (Activity)_4030, Tree Top City - Level 1_2000, Chest Sorter (Assessment)_4035, Happy Camel_4080, Crystals Rule_3021, Cauldron Filler (Assessment)_4030, Pan Balance_2010, Scrub-A-Dub_2000, Happy Camel_2083, Leaf Leader_4020, Chest Sorter (Assessment)_2030, Dino Drink_2000, Cart Balancer (Assessment)_4040, Flower Waterer (Activity)_4030, Bird Measurer (Assessment)_2030, Pirate's Tale_2000,
Pan Balance_4035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_4030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Air Show_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_4035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chow Time_3021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_4010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_3021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_2025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crystals Rule_3120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Dive_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicken Balancer (Activity)_4035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_4040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_4040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Flower Waterer (Activity)_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fireworks (Activity)_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Leaf Leader_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Flower Waterer (Activity)_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crystals Rule_4010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crystals Rule_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sandcastle Builder (Activity)_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_2083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chest Sorter (Assessment)_3021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cauldron Filler (Assessment)_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Watering Hole (Activity)_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chow Time_4030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_4045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chow Time_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Watering Hole (Activity)_2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_4025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chow Time_4010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scrub-A-Dub_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_4035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_3121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Air Show_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Leaf Leader_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicken Balancer (Activity)_4030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Watering Hole (Activity)_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_3121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Leaf Leader_3021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scrub-A-Dub_2030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Egg Dropper (Activity)_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sandcastle Builder (Activity)_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_4031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chow Time_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pan Balance_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Heavy Heavier Heaviest_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_3021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_4035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Dive_3121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pan Balance_4025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_2075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chest Sorter (Assessment)_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pan Balance_4100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pan Balance_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_3120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Egg Dropper (Activity)_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Flower Waterer (Activity)_3110,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
Sandcastle Builder (Activity)_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_4100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Watering Hole (Activity)_5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_3121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_4095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cauldron Filler (Assessment)_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Welcome to Lost Lagoon!_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bug Measurer (Activity)_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Egg Dropper (Activity)_4025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chest Sorter (Assessment)_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Air Show_3021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cart Balancer (Assessment)_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fireworks (Activity)_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fireworks (Activity)_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cart Balancer (Assessment)_3121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scrub-A-Dub_2050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fireworks (Activity)_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cauldron Filler (Assessment)_4040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chow Time_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rulers_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Dive_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chow Time_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Treasure Map_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ordering Spheres_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cauldron Filler (Assessment)_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pan Balance_4030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Air Show_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Dive_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Leaf Leader_4095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_4040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chow Time_3121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scrub-A-Dub_3120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bottle Filler (Activity)_2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cart Balancer (Assessment)_4100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chow Time_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Flower Waterer (Activity)_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cart Balancer (Assessment)_2030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Air Show_2070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bug Measurer (Activity)_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bug Measurer (Activity)_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cart Balancer (Assessment)_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chow Time_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cart Balancer (Assessment)_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Leaf Leader_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_3121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bottle Filler (Activity)_4035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Leaf Leader_2070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Monkeys_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_4030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Egg Dropper (Activity)_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scrub-A-Dub_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_2025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chest Sorter (Assessment)_4100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Leaf Leader_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bug Measurer (Activity)_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Leaf Leader_3120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_4035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Leaf Leader_2060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pan Balance_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cart Balancer (Assessment)_3021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cauldron Filler (Assessment)_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bottle Filler (Activity)_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chest Sorter (Assessment)_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cart Balancer (Assessment)_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scrub-A-Dub_2040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_2080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Watering Hole (Activity)_4021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_4095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bottle Filler (Activity)_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_3021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crystal Caves - Level 1_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Watering Hole (Activity)_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chest Sorter (Assessment)_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bug Measurer (Activity)_4035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Air Show_4010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chest Sorter (Assessment)_4025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bug Measurer (Activity)_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicken Balancer (Activity)_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chow Time_2030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pan Balance_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tree Top City - Level 2_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Air Show_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Egg Dropper (Activity)_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crystals Rule_3121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Dive_3120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scrub-A-Dub_2080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cart Balancer (Assessment)_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cauldron Filler (Assessment)_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chest Sorter (Assessment)_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_2070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Egg Dropper (Activity)_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_3021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bottle Filler (Activity)_2030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bottle Filler (Activity)_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fireworks (Activity)_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cart Balancer (Assessment)_2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicken Balancer (Activity)_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Leaf Leader_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Dive_4010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cart Balancer (Assessment)_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Dive_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_4235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cauldron Filler (Assessment)_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Egg Dropper (Activity)_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bottle Filler (Activity)_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Dive_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sandcastle Builder (Activity)_2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Air Show_4100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Leaf Leader_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_4030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Dive_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Flower Waterer (Activity)_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Air Show_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pan Balance_3120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cauldron Filler (Assessment)_2030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicken Balancer (Activity)_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bottle Filler (Activity)_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scrub-A-Dub_3121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_3120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fireworks (Activity)_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chest Sorter (Assessment)_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_4010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Air Show_2030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crystal Caves - Level 3_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_3120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Dive_3021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Flower Waterer (Activity)_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicken Balancer (Activity)_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sandcastle Builder (Activity)_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Watering Hole (Activity)_4025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chest Sorter (Assessment)_4030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scrub-A-Dub_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cart Balancer (Assessment)_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pan Balance_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crystals Rule_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sandcastle Builder (Activity)_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crystals Rule_4050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crystals Rule_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bug Measurer (Activity)_4025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chest Sorter (Assessment)_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fireworks (Activity)_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_4220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_4010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Flower Waterer (Activity)_4022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sandcastle Builder (Activity)_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_4025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sandcastle Builder (Activity)_4030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cauldron Filler (Assessment)_3021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Honey Cake_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chow Time_4035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Watering Hole (Activity)_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chest Sorter (Assessment)_3121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pan Balance_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Egg Dropper (Activity)_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Leaf Leader_2075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cauldron Filler (Assessment)_3121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cauldron Filler (Assessment)_4035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Egg Dropper (Activity)_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bug Measurer (Activity)_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crystals Rule_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Dive_2030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_3121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_2025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicken Balancer (Activity)_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chow Time_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cauldron Filler (Assessment)_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scrub-A-Dub_2083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Air Show_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cart Balancer (Assessment)_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_2030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bottle Filler (Activity)_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_4030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chow Time_3120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_2080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pan Balance_2030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crystals Rule_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_3120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scrub-A-Dub_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scrub-A-Dub_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bottle Filler (Activity)_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crystals Rule_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cauldron Filler (Assessment)_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_4045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chest Sorter (Assessment)_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Costume Box_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chest Sorter (Assessment)_4080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pan Balance_3121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sandcastle Builder (Activity)_4035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Air Show_2060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sandcastle Builder (Activity)_4021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_2030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pan Balance_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sandcastle Builder (Activity)_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_4030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_3120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cart Balancer (Assessment)_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mushroom Sorter (Assessment)_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Dive_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Watering Hole (Activity)_5010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scrub-A-Dub_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Dive_2060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crystals Rule_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bubble Bath_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_3110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Leaf Leader_2030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cauldron Filler (Assessment)_4025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_4070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Happy Camel_4020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Balancing Act_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Drink_2030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicken Balancer (Activity)_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dino Dive_2070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_3010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Slop Problem_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Air Show_4110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Magma Peak - Level 2_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Crystal Caves - Level 2_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bird Measurer (Assessment)_4110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Star Sorting_3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tree Top City - Level 3_2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0